# MyAnimeStats

Explore insightful statistics about a [MyAnimeList](https://myanimelist.net/) user list.

### List Import

In [ ]:
import httpx
from pathlib import Path
from rich import print
from src.jupyter_utils import *
from src.user_list import UserList
from src.actions import get_user_animes
from src.franchises import get_user_franchises

data = Path('data')
user_list_xml_path = data / 'animelist.xml.gz'
# anime_db_path = data / 'anime_db.parquet'
anime_db_path = data / 'anime_db.franchise.parquet'
# manga_db_path = data / 'manga_db.parquet'
character_db_path = data / 'character_db.parquet'
people_db_path = data / 'people_db.parquet'

cache = data / 'cache'
cache.mkdir(exist_ok=True)
user_name_cache = cache / 'user_name.txt'
user_list_cache = cache / 'user_list.parquet'

if not anime_db_path.exists():
	raise Exception(f"Missing anime_db {anime_db_path}")

def get_user_name():
	if user_name_cache.exists():
		print(f"[green]Loading user_name from cache {user_name_cache}[/green]")
		return user_name_cache.read_text().strip()
	print(f"[red]Failed to load user_name from cache {user_name_cache}[/red]")

	user = input('Enter your MAL user_name: ')
	if user != '':
		print(f"[green]Got user_name from user input, saving to cache {user_name_cache}[/green]") 
		user_name_cache.write_text(user)
		return user
	print("[red]Failed to get user_name from user input[/red]")

	return None

async def get_user_list():
	if user_list_cache.exists():
		print(f"[green]Loading user_list from cache {user_list_cache}[/green]")
		return pl.read_parquet(user_list_cache)
	print(f"[red]Failed to load user_list from cache {user_list_cache}[/red]")

	if user_list_xml_path.exists():
		print(f"[green]Loading user_list from MAL export {user_list_xml_path}[/green]")
		user_list = UserList.from_xml(user_list_xml_path)
		# user_list.write_parquet(user_list_cache)
		return user_list
	print(f"[red]Failed to load user_list from MAL export {user_list_xml_path}[/red]")

	user_name = get_user_name()
	if user_name:
		print(f"[green]Scraping user_list from MAL user_name {user_name}[/green]")
		async with httpx.AsyncClient() as client:
			user_list = await UserList.from_user_name(client, user_name)
			user_list.write_parquet(user_list_cache)
			return user_list
	print("[red]Failed to get user_name to scrape user_list[/red]")

	raise Exception(f"Failed to get user_list, need {user_list_xml_path} or user_name")

user_list = await get_user_list()

user_animes = get_user_animes(user_list, anime_db_path)
describe(user_animes, "User Animes")

user_franchises = get_user_franchises(user_animes)
print(f"[green]Found {user_franchises.height} franchises[/green]")
describe(user_franchises, "User Franchises")

### List Analysis

In [ ]:
import tzlocal
from src.actions import get_stats

# Get local timezone
local_tz = tzlocal.get_localzone_name()
print(f"[green]Detected local timezone: {local_tz}[/green]")

stats = get_stats(user_animes, user_franchises, local_tz)

for name, stat in stats.items():
	print_df(stat, name)